<a href="https://colab.research.google.com/github/winnerking2010/AgentGPT/blob/main/colab0325%E5%BE%AE%E8%B0%83DEEPSEEK%E8%92%B8%E9%A6%8Fllama8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

一、加载unsloth

In [ ]:
!pip install unsloth

**二、下载unsloth 4bit 量化大模型和分词器**

In [ ]:
#减少序列长度从2048到1024
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    max_seq_length = 1024,
    dtype = None,
    load_in_4bit = True,)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


**三、将 LoRA 矩阵添加到预训练 LLM中，这将有助于微调模型的响应**

In [ ]:
from unsloth import FastLanguageModel

model = FastLanguageModel.get_peft_model(
    model,
    r=64,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_alpha=32,
    lora_dropout=0.05,  # 可设置为任意值，但建议设为0以优化性能[2](@ref)
    bias="none",        # 建议设为"none"以优化性能[5](@ref)
    use_gradient_checkpointing="unsloth",  # 支持超长上下文训练[2](@ref)
    random_state=3977,
    use_rslora=False,  # Unsloth也支持秩稳定LoRA[5](@ref)
    loftq_config=None   # 可选LoftQ配置
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.3.18 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


**四、Prompt的结构**

In [ ]:
alpaca_prompt="""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{}
### Input:
{}
### Response:
{}"""

**五、创建了一个函数，它将正确地构建 alpaca_prompt中的所有数据**

In [ ]:
EOS_TOKEN = tokenizer.eos_token  # 标记序列结束，防止生成无限延伸[1](@ref)

def formatting_prompts_func(examples):
    """将对话数据转换为带EOS标记的输入格式"""
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []

    # 按模板生成带EOS的输入文本
    for instruction, input_text, output_text in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(
            instruction=instruction,
            input=input_text,
            output=output_text
        ) + EOS_TOKEN  # 必须添加EOS_TOKEN[1](@ref)
        texts.append(text)

    return {"text": texts}

**六、加载将用于微调模型的数据集**

In [ ]:
# 添加类型提示（可选）
from datasets import DatasetDict

def load_and_preprocess():
    """加载并预处理Alpaca数据集"""
    dataset = load_dataset(
        "yahma/alpaca-cleaned",
        split="train"
    )
    return dataset.map(
        formatting_prompts_func,
        batched=True,
    )

**七、使用 SFTTrainer和 hyperparameter 初始化trainer**

In [ ]:
!pip install trl

In [ ]:
!pip install datasets

In [ ]:
#代码持续报错 GPU溢出 超出内存？？
import datasets
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)

from trl import SFTTrainer  # [1,3,4](@ref)
from transformers import TrainingArguments  # [1,5,6](@ref)
from unsloth import is_bfloat16_supported  # [4,7](@ref)

trainer = SFTTrainer(
    model=model,  # [1,3,5](@ref)
    tokenizer=tokenizer,  # [1,3,5](@ref)
    train_dataset=datasets,  # [1,3,5](@ref)
    datasets_text_field="text",  # [1,3,5](@ref)
    max_seq_length=max_seq_length,  # [1,3,5](@ref)
    datasets_num_proc=2,  # [1,3,5](@ref)
    packing=False,  # [3,5](@ref)
    args=TrainingArguments(
        per_device_train_batch_size=2,  # [1,5,6](@ref)
        gradient_accumulation_steps=4,  # [1,5,6](@ref)
        warmup_steps=5,  # [1,5,6](@ref)
        max_steps=120,  # [1,5](@ref)
        learning_rate=2e-4,  # [1,5,6](@ref)
        fp16=not is_bfloat16_supported(),  # [1,4,7](@ref)
        bf16=is_bfloat16_supported(),  # [1,4,7](@ref)
        logging_steps=1,  # [1,5,6](@ref)
        optim="adamw_8bit",  # [1,5,6](@ref)
        weight_decay=0.01,  # [1,5,6](@ref)
        lr_scheduler_type="linear",  # [1,5,6](@ref)
        seed=3407,  # [1,3,5](@ref)
        output_dir="outputs",  # [1,3,5](@ref)
        report_to="none",  # [1,3,5](@ref)
    ),
)

In [ ]:
# 报错！！
import datasets
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# 加载模型和分词器
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)

# 加载数据集，这里以加载一个示例数据集为例
train_dataset = datasets.load_dataset("wikitext", "wikitext-2-raw-v1", split="train")

# 对数据集进行简单处理，假设数据集中的文本字段名为 "text"
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=2048)

tokenized_dataset = train_dataset.map(tokenize_function, batched=True)

# 定义训练参数
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=120,
    learning_rate=2e-4,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs",
    report_to="none",
)

# 创建 SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset,
    max_seq_length=2048,
    packing=False,
    args=training_args,
)

# 开始训练
trainer.train()

In [ ]:
# TypeError: LlamaForCausalLM.__init__() got an unexpected keyword argument 'llm_int8_enable_fp32_cpu_offload'
import torch
import datasets
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# 定义自定义的 device_map
device_map = {
    "": 0  # 假设只有一个GPU，将整个模型分配到该GPU
}

# 加载模型和分词器，设置 llm_int8_enable_fp32_cpu_offload 和 device_map
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
    llm_int8_enable_fp32_cpu_offload=True,
    device_map=device_map
)

# 加载数据集，这里以加载一个示例数据集为例
train_dataset = datasets.load_dataset("wikitext", "wikitext-2-raw-v1", split="train")

# 对数据集进行简单处理，假设数据集中的文本字段名为 "text"
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=2048)

tokenized_dataset = train_dataset.map(tokenize_function, batched=True)

# 定义训练参数
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=120,
    learning_rate=2e-4,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs",
    report_to="none",
)

# 创建 SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset,
    max_seq_length=2048,
    packing=False,
    args=training_args,
)

# 开始训练
trainer.train()

In [ ]:
#OutOfMemoryError: CUDA out of memory. Tried to allocate 28.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 10.12 MiB is free. Process 39053 has 14.73 GiB memory in use. Of the allocated memory 14.59 GiB is allocated by PyTorch, and 6.22 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
import torch
import datasets
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# 定义自定义的 device_map
device_map = {
    "": 0  # 假设只有一个GPU，将整个模型分配到该GPU
}

# 加载模型和分词器
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
    device_map=device_map
)

# 加载数据集，这里以加载一个示例数据集为例
train_dataset = datasets.load_dataset("wikitext", "wikitext-2-raw-v1", split="train")

# 对数据集进行简单处理，假设数据集中的文本字段名为 "text"
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=2048)

tokenized_dataset = train_dataset.map(tokenize_function, batched=True)

# 定义训练参数
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=120,
    learning_rate=2e-4,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs",
    report_to="none",
)

# 创建 SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset,
    max_seq_length=2048,
    packing=False,
    args=training_args,
)

# 开始训练
trainer.train()

In [ ]:
#OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 10.12 MiB is free. Process 39053 has 14.73 GiB memory in use. Of the allocated memory 14.59 GiB is allocated by PyTorch, and 6.22 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
import torch
import datasets
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# 定义自定义的 device_map
device_map = {
    "": 0  # 假设只有一个GPU，将整个模型分配到该GPU
}

# 加载模型和分词器
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
    device_map=device_map
)

# 加载数据集，这里以加载一个示例数据集为例
train_dataset = datasets.load_dataset("wikitext", "wikitext-2-raw-v1", split="train")

# 对数据集进行简单处理，假设数据集中的文本字段名为 "text"
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=2048)

tokenized_dataset = train_dataset.map(tokenize_function, batched=True)

# 定义训练参数，减小批量大小，增加梯度累积步数
training_args = TrainingArguments(
    per_device_train_batch_size=1,  # 减小批量大小
    gradient_accumulation_steps=8,  # 增加梯度累积步数
    warmup_steps=5,
    max_steps=120,
    learning_rate=2e-4,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs",
    report_to="none",
)

# 创建 SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset,
    max_seq_length=2048,
    packing=False,
    args=training_args,
)

# 开始训练
try:
    trainer.train()
except RuntimeError as e:
    if "CUDA out of memory" in str(e):
        print("GPU 内存不足，请尝试进一步调整参数。")
        torch.cuda.empty_cache()  # 清除缓存
    else:
        print(f"发生其他错误: {e}")

In [ ]:
#优化控制GPU在14G以内
# ValueError: You cannot perform fine-tuning on purely quantized models. Please attach trainable adapters on top of the quantized model to correctly perform fine-tuning. Please see: https://huggingface.co/docs/transformers/peft for more details
import torch
import datasets
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# 定义自定义的 device_map
device_map = {
    "": 0  # 假设只有一个GPU，将整个模型分配到该GPU
}

# 加载模型和分词器
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
    device_map=device_map
)

# 加载数据集，这里以加载一个示例数据集为例
train_dataset = datasets.load_dataset("wikitext", "wikitext-2-raw-v1", split="train")

# 对数据集进行简单处理，假设数据集中的文本字段名为 "text"
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=2048)

# 以流式方式处理数据集，减少内存占用
tokenized_dataset = train_dataset.map(tokenize_function, batched=True, num_proc=1, remove_columns=train_dataset.column_names)

# 定义训练参数，进一步减小批量大小，增加梯度累积步数
training_args = TrainingArguments(
    per_device_train_batch_size=1,  # 进一步减小批量大小
    gradient_accumulation_steps=16,  # 进一步增加梯度累积步数
    warmup_steps=5,
    max_steps=120,
    learning_rate=2e-4,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=1,
    optim="adamw_8bit",  # 使用 8bit 的 AdamW 优化器
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs",
    report_to="none",
    save_steps=10000  # 减少保存频率，避免保存中间模型占用过多内存
)

# 创建 SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset,
    max_seq_length=2048,
    packing=False,
    args=training_args,
)

# 开始训练
try:
    trainer.train()
except RuntimeError as e:
    if "CUDA out of memory" in str(e):
        print("GPU 内存不足，请尝试进一步调整参数。")
        torch.cuda.empty_cache()  # 清除缓存
    else:
        print(f"发生其他错误: {e}")


**八、加载 peft**

In [ ]:
!pip install peft

In [ ]:
import torch
import datasets
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from peft import LoraConfig, get_peft_model

# 定义自定义的 device_map
device_map = {
    "": 0  # 假设只有一个GPU，将整个模型分配到该GPU
}

# 加载模型和分词器
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
    device_map=device_map
)

# 为模型添加 LoRA 适配器
config = LoraConfig(
    r=8,  # LoRA 秩
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # 需要调整的目标模块
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, config)

# 加载数据集，这里以加载一个示例数据集为例
train_dataset = datasets.load_dataset("wikitext", "wikitext-2-raw-v1", split="train")

# 对数据集进行简单处理，假设数据集中的文本字段名为 "text"
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=2048)

# 以流式方式处理数据集，减少内存占用
tokenized_dataset = train_dataset.map(tokenize_function, batched=True, num_proc=1, remove_columns=train_dataset.column_names)

# 定义训练参数，进一步减小批量大小，增加梯度累积步数
training_args = TrainingArguments(
    per_device_train_batch_size=1,  # 进一步减小批量大小
    gradient_accumulation_steps=16,  # 进一步增加梯度累积步数
    warmup_steps=5,
    max_steps=120,
    learning_rate=2e-4,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=1,
    optim="adamw_8bit",  # 使用 8bit 的 AdamW 优化器
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs",
    report_to="none",
    save_steps=10000  # 减少保存频率，避免保存中间模型占用过多内存
)

# 创建 SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset,
    max_seq_length=2048,
    packing=False,
    args=training_args,
)

# 开始训练
try:
    trainer.train()
    # 保存微调后的模型
    model.save_pretrained("outputs")
except RuntimeError as e:
    if "CUDA out of memory" in str(e):
        print("GPU 内存不足，请尝试进一步调整参数。")
        torch.cuda.empty_cache()  # 清除缓存
    else:
        print(f"发生其他错误: {e}")

#OutOfMemoryError: CUDA out of memory. Tried to allocate 28.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 28.12 MiB is free. Process 3588 has 14.71 GiB memory in use. Of the allocated memory 14.57 GiB is allocated by PyTorch, and 7.33 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)



In [ ]:
import torch
import datasets
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from peft import LoraConfig, get_peft_model

# 定义自定义的 device_map
device_map = {
    "": 0  # 假设只有一个GPU，将整个模型分配到该GPU
}

# 加载模型和分词器
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    max_seq_length=1024,  # 减少序列长度
    dtype=None,
    load_in_4bit=True,
    device_map=device_map
)

# 为模型添加 LoRA 适配器
config = LoraConfig(
    r=4,  # 降低 LoRA 秩
    lora_alpha=8,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, config)

# 加载数据集，这里以加载一个示例数据集为例
train_dataset = datasets.load_dataset("wikitext", "wikitext-2-raw-v1", split="train")

# 对数据集进行简单处理，假设数据集中的文本字段名为 "text"
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=1024)

# 以流式方式处理数据集，减少内存占用
tokenized_dataset = train_dataset.map(tokenize_function, batched=True, num_proc=1, remove_columns=train_dataset.column_names)

# 定义训练参数，进一步减小批量大小，增加梯度累积步数
training_args = TrainingArguments(
    per_device_train_batch_size=1,  # 进一步减小批量大小
    gradient_accumulation_steps=32,  # 进一步增加梯度累积步数
    warmup_steps=5,
    max_steps=120,
    learning_rate=2e-4,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs",
    report_to="none",
    save_steps=10000  # 减少保存频率，避免保存中间模型占用过多内存
)

# 创建 SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset,
    max_seq_length=1024,
    packing=False,
    args=training_args,
)

# 开始训练
try:
    trainer.train()
    # 保存微调后的模型
    model.save_pretrained("outputs")
except RuntimeError as e:
    if "CUDA out of memory" in str(e):
        print("GPU 内存不足，请尝试进一步调整参数。")
        torch.cuda.empty_cache()  # 清除缓存
    else:
        print(f"发生其他错误: {e}")

#OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 8.12 MiB is free. Process 3588 has 14.73 GiB memory in use. Of the allocated memory 14.59 GiB is allocated by PyTorch, and 11.33 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
#第六次优化 控制内存
import torch
import datasets
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from peft import LoraConfig, get_peft_model

# 定义自定义的 device_map
device_map = {
    "": 0  # 假设只有一个GPU，将整个模型分配到该GPU
}

# 加载模型和分词器，进一步降低 max_seq_length
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    max_seq_length=512,
    dtype=None,
    load_in_4bit=True,
    device_map=device_map
)

# 为模型添加 LoRA 适配器，进一步降低 r 值
config = LoraConfig(
    r=2,  # 降低 LoRA 秩
    lora_alpha=4,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, config)

# 加载数据集，这里以加载一个示例数据集为例
train_dataset = datasets.load_dataset("wikitext", "wikitext-2-raw-v1", split="train")

# 对数据集进行简单处理，假设数据集中的文本字段名为 "text"
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512)

# 以流式方式处理数据集，减少内存占用
tokenized_dataset = train_dataset.map(tokenize_function, batched=True, num_proc=1, remove_columns=train_dataset.column_names)

# 定义训练参数，进一步减小批量大小和增加梯度累积步数
training_args = TrainingArguments(
    per_device_train_batch_size=1,  # 保持小批量
    gradient_accumulation_steps=64,  # 进一步增加梯度累积步数
    warmup_steps=5,
    max_steps=120,
    learning_rate=2e-4,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=1,
    optim="adamw_8bit",  # 使用 8bit 的 AdamW 优化器
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs",
    report_to="none",
    save_steps=10000  # 减少保存频率，避免保存中间模型占用过多内存
)

# 创建 SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset,
    max_seq_length=512,
    packing=False,
    args=training_args,
)

# 开始训练
try:
    for step in range(training_args.max_steps):
        trainer.train_step()
        if step % 10 == 0:  # 每 10 步清除一次缓存
            torch.cuda.empty_cache()
    # 保存微调后的模型
    model.save_pretrained("outputs")
except RuntimeError as e:
    if "CUDA out of memory" in str(e):
        print("GPU 内存不足，请尝试进一步调整参数。")
        torch.cuda.empty_cache()  # 清除缓存
    else:
        print(f"发生其他错误: {e}")

#AttributeError: 'UnslothSFTTrainer' object has no attribute 'train_step'


**九、加载训练数据 迭代优化**

In [ ]:
import torch
import datasets
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from peft import LoraConfig, get_peft_model

# 定义自定义的 device_map
device_map = {
    "": 0  # 假设只有一个GPU，将整个模型分配到该GPU
}

# 加载模型和分词器，进一步降低 max_seq_length
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    max_seq_length=512,
    dtype=None,
    load_in_4bit=True,
    device_map=device_map
)

# 为模型添加 LoRA 适配器，进一步降低 r 值
config = LoraConfig(
    r=2,  # 降低 LoRA 秩
    lora_alpha=4,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, config)

# 加载数据集，这里以加载一个示例数据集为例
train_dataset = datasets.load_dataset("wikitext", "wikitext-2-raw-v1", split="train")

# 对数据集进行简单处理，假设数据集中的文本字段名为 "text"
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512)

# 以流式方式处理数据集，减少内存占用
tokenized_dataset = train_dataset.map(tokenize_function, batched=True, num_proc=1, remove_columns=train_dataset.column_names)

# 定义训练参数，进一步减小批量大小和增加梯度累积步数
training_args = TrainingArguments(
    per_device_train_batch_size=1,  # 保持小批量
    gradient_accumulation_steps=64,  # 进一步增加梯度累积步数
    warmup_steps=5,
    max_steps=120,
    learning_rate=2e-4,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=1,
    optim="adamw_8bit",  # 使用 8bit 的 AdamW 优化器
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs",
    report_to="none",
    save_steps=10000  # 减少保存频率，避免保存中间模型占用过多内存
)

# 创建 SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset,
    max_seq_length=512,
    packing=False,
    args=training_args,
)

# 开始训练
try:
    trainer.train()
    # 每训练一定步数后清除缓存
    torch.cuda.empty_cache()
    # 保存微调后的模型
    model.save_pretrained("outputs")
except RuntimeError as e:
    if "CUDA out of memory" in str(e):
        print("GPU 内存不足，请尝试进一步调整参数。")
        torch.cuda.empty_cache()  # 清除缓存
    else:
        print(f"发生其他错误: {e}")

#OutOfMemoryError: CUDA out of memory. Tried to allocate 28.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 26.12 MiB is free. Process 89773 has 14.71 GiB memory in use. Of the allocated memory 14.57 GiB is allocated by PyTorch, and 6.08 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


In [ ]:
#第10次优化  控制内存
import torch
import datasets
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from peft import LoraConfig, get_peft_model

# 定义自定义的 device_map
device_map = {
    "": 0  # 假设只有一个GPU，将整个模型分配到该GPU
}

# 加载模型和分词器，再次降低 max_seq_length
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    max_seq_length=256,
    dtype=None,
    load_in_4bit=True,
    device_map=device_map
)

# 为模型添加 LoRA 适配器，进一步降低 r 和 lora_alpha 值
config = LoraConfig(
    r=1,  # 再次降低 LoRA 秩
    lora_alpha=2,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, config)

# 加载数据集，这里以加载一个示例数据集为例
train_dataset = datasets.load_dataset("wikitext", "wikitext-2-raw-v1", split="train")

# 对数据集进行简单处理，假设数据集中的文本字段名为 "text"
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=256)

# 以流式方式处理数据集，减少内存占用
tokenized_dataset = train_dataset.map(tokenize_function, batched=True, num_proc=1, remove_columns=train_dataset.column_names)

# 定义训练参数，进一步增加梯度累积步数
training_args = TrainingArguments(
    per_device_train_batch_size=1,  # 保持小批量
    gradient_accumulation_steps=128,  # 进一步增加梯度累积步数
    warmup_steps=5,
    max_steps=120,
    learning_rate=2e-4,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=1,
    optim="adamw_8bit",  # 使用 8bit 的 AdamW 优化器
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs",
    report_to="none",
    save_steps=10000  # 减少保存频率，避免保存中间模型占用过多内存
)

# 创建 SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset,
    max_seq_length=256,
    packing=False,
    args=training_args,
)

# 开始训练
try:
    trainer.train()
    # 每训练一定步数后清除缓存
    torch.cuda.empty_cache()
    # 保存微调后的模型
    model.save_pretrained("outputs")
except RuntimeError as e:
    if "CUDA out of memory" in str(e):
        print("GPU 内存不足，请尝试进一步调整参数。")
        torch.cuda.empty_cache()  # 清除缓存
    else:
        print(f"发生其他错误: {e}")

#发生其他错误: element 0 of tensors does not require grad and does not have a grad_fn


In [ ]:
#第11次优化  梯度反向更新
import torch
import datasets
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from peft import LoraConfig, get_peft_model

# 定义自定义的 device_map
device_map = {
    "": 0  # 假设只有一个GPU，将整个模型分配到该GPU
}

# 加载模型和分词器，再次降低 max_seq_length
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    max_seq_length=256,
    dtype=None,
    load_in_4bit=True,
    device_map=device_map
)

# 为模型添加 LoRA 适配器，进一步降低 r 和 lora_alpha 值
config = LoraConfig(
    r=1,  # 再次降低 LoRA 秩
    lora_alpha=2,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, config)

# 检查并确保只有 LoRA 适配器的参数需要梯度
for name, param in model.named_parameters():
    if 'lora' not in name:
        param.requires_grad = False

# 打印可训练参数，用于调试
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

# 加载数据集，这里以加载一个示例数据集为例
train_dataset = datasets.load_dataset("wikitext", "wikitext-2-raw-v1", split="train")

# 对数据集进行简单处理，假设数据集中的文本字段名为 "text"
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=256)

# 以流式方式处理数据集，减少内存占用
tokenized_dataset = train_dataset.map(tokenize_function, batched=True, num_proc=1, remove_columns=train_dataset.column_names)

# 定义训练参数，进一步增加梯度累积步数
training_args = TrainingArguments(
    per_device_train_batch_size=1,  # 保持小批量
    gradient_accumulation_steps=128,  # 进一步增加梯度累积步数
    warmup_steps=5,
    max_steps=120,
    learning_rate=2e-4,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=1,
    optim="adamw_8bit",  # 使用 8bit 的 AdamW 优化器
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs",
    report_to="none",
    save_steps=10000  # 减少保存频率，避免保存中间模型占用过多内存
)

# 创建 SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset,
    max_seq_length=256,
    packing=False,
    args=training_args,
)

# 开始训练
try:
    trainer.train()
    # 每训练一定步数后清除缓存
    torch.cuda.empty_cache()
    # 保存微调后的模型
    model.save_pretrained("outputs")
except RuntimeError as e:
    if "CUDA out of memory" in str(e):
        print("GPU 内存不足，请尝试进一步调整参数。")
        torch.cuda.empty_cache()  # 清除缓存
    else:
        print(f"发生其他错误: {e}")


In [ ]:
#第12次
import torch
import datasets
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from peft import LoraConfig, get_peft_model

# 定义自定义的 device_map
device_map = {
    "": 0  # 假设只有一个GPU，将整个模型分配到该GPU
}

# 加载模型和分词器，再次降低 max_seq_length
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    max_seq_length=256,
    dtype=None,
    load_in_4bit=True,
    #PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
    device_map=device_map
)

# 为模型添加 LoRA 适配器，进一步降低 r 和 lora_alpha 值
config = LoraConfig(
    r=1,  # 再次降低 LoRA 秩
    lora_alpha=2,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, config)

# 确保只有 LoRA 适配器的参数需要梯度
for name, param in model.named_parameters():
    if 'lora' not in name:
        param.requires_grad = False

# 过滤出可训练的参数
trainable_params = [param for param in model.parameters() if param.requires_grad]

# 定义训练参数，进一步增加梯度累积步数
training_args = TrainingArguments(
    per_device_train_batch_size=1,  # 保持小批量
    gradient_accumulation_steps=128,  # 进一步增加梯度累积步数
    warmup_steps=5,
    max_steps=120,
    learning_rate=2e-4,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=1,
    optim="adamw_8bit",  # 使用 8bit 的 AdamW 优化器
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs",
    report_to="none",
    save_steps=5000  # 减少保存频率，避免保存中间模型占用过多内存
)

# 创建 SFTTrainer，明确传递可训练参数
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    max_seq_length=256,
    packing=False,
    args=training_args,
    trainable_params=trainable_params  # 传递可训练参数
)

# 加载数据集，这里以加载一个示例数据集为例
train_dataset = datasets.load_dataset("wikitext", "wikitext-2-raw-v1", split="train")

# 对数据集进行简单处理，假设数据集中的文本字段名为 "text"
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=256)

# 以流式方式处理数据集，减少内存占用
tokenized_dataset = train_dataset.map(tokenize_function, batched=True, num_proc=1, remove_columns=train_dataset.column_names)

# 开始训练
try:
    trainer.train()
    # 每训练一定步数后清除缓存
    torch.cuda.empty_cache()
    # 保存微调后的模型
    model.save_pretrained("deepseekoutputsllama8B")
except RuntimeError as e:
    if "CUDA out of memory" in str(e):
        print("GPU 内存不足，请尝试进一步调整参数。")
        torch.cuda.empty_cache()  # 清除缓存
    else:
        print(f"发生其他错误: {e}")

#OutOfMemoryError: CUDA out of memory. Tried to allocate 28.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 20.12 MiB is free. Process 123549 has 14.72 GiB memory in use. Of the allocated memory 14.56 GiB is allocated by PyTorch, and 18.41 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
